Predict cost of used cars - Preprocessing

In [1]:
import warnings
warnings.filterwarnings('ignore')

import random
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, cross_validate
le = LabelEncoder()

import sklearn
import xgboost as xgb
import lightgbm as lgbm
import scipy
random.seed(20)

Using TensorFlow backend.


In [2]:
train_data = pd.read_excel('Participants_Data_Used_Cars/Data_Train.xlsx')
test_data = pd.read_excel('Participants_Data_Used_Cars/Data_Test.xlsx')

# Experiment 1 - 0.9053

## Preprocessing

In [4]:
X_train = train_data.drop(columns='Price')
y_train = train_data['Price']

X_test = test_data

In [5]:
X_train['Engine'] = X_train['Engine'].str[:-3].replace(np.nan, -1).astype('int').replace(-1, np.nan)
X_test['Engine'] = X_test['Engine'].str[:-3].replace(np.nan, -1).astype('int').replace(-1, np.nan)

In [6]:
X_train['Power'] = X_train['Power'].str[:-4].replace(np.nan, -1).replace('null', -1).astype('float').replace(-1, np.nan)
X_test['Power'] = X_test['Power'].str[:-4].replace(np.nan, -1).replace('null', -1).astype('float').replace(-1, np.nan)

### Drop columns

In [7]:
X_train = X_train.drop(columns=['New_Price', 'Mileage'])
X_test = X_test.drop(columns=['New_Price', 'Mileage'])

In [8]:
X_train.isnull().sum()

Name                   0
Location               0
Year                   0
Kilometers_Driven      0
Fuel_Type              0
Transmission           0
Owner_Type             0
Engine                36
Power                143
Seats                 42
dtype: int64

In [9]:
X_test.isnull().sum()

Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Engine               10
Power                32
Seats                11
dtype: int64

### Impute missing values

In [11]:
def fill_engine(x):
    return x.sum()/len(x)

X_train['Engine'] = X_train.groupby(['Name'])['Engine'].transform(lambda x: x.fillna(fill_engine(x)))

values = X_test[X_test['Engine'].isnull()]['Name'].values
X_test['Engine'].ix[X_test['Engine'].isnull()] = list(X_train.groupby('Name')['Engine'].mean()[values].values)

X_test['Engine'] = X_test['Engine'].fillna(X_train['Engine'].mean())

In [12]:
def fill_power(x):
    return x.sum()/len(x)

X_train['Power'] = X_train.groupby(['Name'])['Power'].transform(lambda x: x.fillna(fill_power(x)))

values = X_test[X_test['Power'].isnull()]['Name'].values
X_test['Power'].ix[X_test['Power'].isnull()] = list(X_train.groupby('Name')['Power'].mean()[values].values)

X_test['Power'] = X_test['Power'].fillna(X_train['Power'].mean())

In [13]:
def fill_seats(x, X_train):
    try:
        return x.mode()[0]
    except:
        return X_train['Seats'].mode()[0]
    
X_train['Seats'] = X_train.groupby(['Name'])['Seats'].transform(lambda x: x.fillna(fill_seats(x, X_train)))
X_test['Seats'] = X_test['Seats'].fillna(X_train['Seats'].mode()[0])

### Label Encoding

In [16]:
object_columns = ['Name', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type']

for col in object_columns:
    label_encoder = le.fit(list(X_train[col]) + list(X_test[col]))
    X_train[col] = label_encoder.transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])

## Modeling

### XGBoost

In [21]:
XGB_model = xgb.XGBRegressor()

In [22]:
XGB_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### Predict and Save

In [23]:
y_pred = XGB_model.predict(X_test)

In [24]:
y_pred

array([ 3.740502 ,  2.8587976, 16.547705 , ...,  3.5477343,  4.9571767,
       20.1998   ], dtype=float32)

In [25]:
y_pred_df = pd.DataFrame({'Price': y_pred})

In [29]:
y_pred_df.to_excel('Submissions/1_Predictions.xlsx', index=False)

0.9053

# Experiment 2 - 0.9040

## Modeling

### XGBoost

In [19]:
XGB_model = xgb.XGBRegressor(n_estimators = 100, 
                            max_depth = 3,
                            colsample_bylevel = 0.9,
                            learning_rate = 0.1,
                            random_state=12)

In [20]:
XGB_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=12,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### Predict and Save

In [21]:
y_pred = XGB_model.predict(X_test)

In [22]:
y_pred

array([ 3.8048656,  2.933293 , 16.270998 , ...,  3.7403195,  5.431561 ,
       19.583801 ], dtype=float32)

In [23]:
y_pred_df = pd.DataFrame({'Price': y_pred})

In [24]:
y_pred_df.to_excel('Submissions/2_Predictions.xlsx', index=False)

0.9040

# Experiment 3 - 0.9056

## Preprocessing

In [74]:
X_train = train_data.drop(columns='Price')
y_train = train_data['Price']

X_test = test_data.copy()

In [75]:
X_train['Engine'] = X_train['Engine'].str[:-3].replace(np.nan, -1).astype('int').replace(-1, np.nan)
X_test['Engine'] = X_test['Engine'].str[:-3].replace(np.nan, -1).astype('int').replace(-1, np.nan)

In [76]:
X_train['Power'] = X_train['Power'].str[:-4].replace(np.nan, -1).replace('null', -1).astype('float').replace(-1, np.nan)
X_test['Power'] = X_test['Power'].str[:-4].replace(np.nan, -1).replace('null', -1).astype('float').replace(-1, np.nan)

In [77]:
X_train['Mileage'] = X_train.Mileage.str.split(expand=True).drop(columns=1).rename(columns={0: 'Mileage'}).Mileage.astype('float')
X_test['Mileage'] = X_test.Mileage.str.split(expand=True).drop(columns=1).rename(columns={0: 'Mileage'}).Mileage.astype('float')

In [78]:
X_train.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.60,998.0,58.16,5.0,NaN
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,NaN
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.20,1199.0,88.70,5.0,8.61 Lakh
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,NaN
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,NaN


### Drop columns

In [79]:
X_train = X_train.drop(columns=['New_Price'])
X_test = X_test.drop(columns=['New_Price'])

In [80]:
X_train.isnull().sum()

Name                   0
Location               0
Year                   0
Kilometers_Driven      0
Fuel_Type              0
Transmission           0
Owner_Type             0
Mileage                2
Engine                36
Power                143
Seats                 42
dtype: int64

In [81]:
X_test.isnull().sum()

Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               0
Engine               10
Power                32
Seats                11
dtype: int64

### Impute missing values

In [82]:
X_train['Mileage'] = X_train.groupby(['Fuel_Type'])['Mileage'].transform(lambda x: x.fillna(x.mean()))

In [83]:
def fill_engine(x):
    return x.sum()/len(x)

X_train['Engine'] = X_train.groupby(['Name'])['Engine'].transform(lambda x: x.fillna(fill_engine(x)))

values = X_test[X_test['Engine'].isnull()]['Name'].values
X_test['Engine'].ix[X_test['Engine'].isnull()] = list(X_train.groupby('Name')['Engine'].mean()[values].values)

X_test['Engine'] = X_test['Engine'].fillna(X_train['Engine'].mean())

In [84]:
def fill_power(x):
    return x.sum()/len(x)

X_train['Power'] = X_train.groupby(['Name'])['Power'].transform(lambda x: x.fillna(fill_power(x)))

values = X_test[X_test['Power'].isnull()]['Name'].values
X_test['Power'].ix[X_test['Power'].isnull()] = list(X_train.groupby('Name')['Power'].mean()[values].values)

X_test['Power'] = X_test['Power'].fillna(X_train['Power'].mean())

In [85]:
def fill_seats(x, X_train):
    try:
        return x.mode()[0]
    except:
        return X_train['Seats'].mode()[0]
    
X_train['Seats'] = X_train.groupby(['Name'])['Seats'].transform(lambda x: x.fillna(fill_seats(x, X_train)))
X_test['Seats'] = X_test['Seats'].fillna(X_train['Seats'].mode()[0])

### Label Encoding

In [86]:
object_columns = ['Name', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type']

for col in object_columns:
    label_encoder = le.fit(list(X_train[col]) + list(X_test[col]))
    X_train[col] = label_encoder.transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])

## Modeling

### XGBoost

In [87]:
XGB_model = xgb.XGBRegressor()

In [88]:
XGB_model.fit(X_train, y_train)

[16:52:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

### Predict and Save

In [89]:
y_pred = XGB_model.predict(X_test)

In [90]:
y_pred

array([ 4.104073 ,  2.9036531, 17.021458 , ...,  3.6415722,  5.267805 ,
       20.498932 ], dtype=float32)

In [91]:
y_pred_df = pd.DataFrame({'Price': y_pred})

In [96]:
y_pred_df.to_excel('Submissions/3_Predictions.xlsx', index=False)

# Experiment 4 - 0.9181

## Preprocessing

In [98]:
X_train = train_data.drop(columns='Price')
y_train = train_data['Price']

X_test = test_data.copy()

In [99]:
X_train['Engine'] = X_train['Engine'].str[:-3].replace(np.nan, -1).astype('int').replace(-1, np.nan)
X_test['Engine'] = X_test['Engine'].str[:-3].replace(np.nan, -1).astype('int').replace(-1, np.nan)

In [100]:
X_train['Power'] = X_train['Power'].str[:-4].replace(np.nan, -1).replace('null', -1).astype('float').replace(-1, np.nan)
X_test['Power'] = X_test['Power'].str[:-4].replace(np.nan, -1).replace('null', -1).astype('float').replace(-1, np.nan)

In [101]:
X_train['Mileage'] = X_train.Mileage.str.split(expand=True).drop(columns=1).rename(columns={0: 'Mileage'}).Mileage.astype('float')
X_test['Mileage'] = X_test.Mileage.str.split(expand=True).drop(columns=1).rename(columns={0: 'Mileage'}).Mileage.astype('float')

In [102]:
X_train.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.60,998.0,58.16,5.0,NaN
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,NaN
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.20,1199.0,88.70,5.0,8.61 Lakh
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,NaN
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,NaN


### Drop columns

In [103]:
X_train = X_train.drop(columns=['New_Price'])
X_test = X_test.drop(columns=['New_Price'])

In [104]:
X_train.isnull().sum()

Name                   0
Location               0
Year                   0
Kilometers_Driven      0
Fuel_Type              0
Transmission           0
Owner_Type             0
Mileage                2
Engine                36
Power                143
Seats                 42
dtype: int64

In [105]:
X_test.isnull().sum()

Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               0
Engine               10
Power                32
Seats                11
dtype: int64

### Impute missing values

In [106]:
X_train['Mileage'] = X_train.groupby(['Fuel_Type'])['Mileage'].transform(lambda x: x.fillna(x.mean()))

In [107]:
def fill_engine(x):
    return x.sum()/len(x)

X_train['Engine'] = X_train.groupby(['Name'])['Engine'].transform(lambda x: x.fillna(fill_engine(x)))

values = X_test[X_test['Engine'].isnull()]['Name'].values
X_test['Engine'].ix[X_test['Engine'].isnull()] = list(X_train.groupby('Name')['Engine'].mean()[values].values)

X_test['Engine'] = X_test['Engine'].fillna(X_train['Engine'].mean())

In [108]:
def fill_power(x):
    return x.sum()/len(x)

X_train['Power'] = X_train.groupby(['Name'])['Power'].transform(lambda x: x.fillna(fill_power(x)))

values = X_test[X_test['Power'].isnull()]['Name'].values
X_test['Power'].ix[X_test['Power'].isnull()] = list(X_train.groupby('Name')['Power'].mean()[values].values)

X_test['Power'] = X_test['Power'].fillna(X_train['Power'].mean())

In [109]:
def fill_seats(x, X_train):
    try:
        return x.mode()[0]
    except:
        return X_train['Seats'].mode()[0]
    
X_train['Seats'] = X_train.groupby(['Name'])['Seats'].transform(lambda x: x.fillna(fill_seats(x, X_train)))
X_test['Seats'] = X_test['Seats'].fillna(X_train['Seats'].mode()[0])

### Label Encoding

In [110]:
object_columns = ['Name', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type']

for col in object_columns:
    label_encoder = le.fit(list(X_train[col]) + list(X_test[col]))
    X_train[col] = label_encoder.transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])

## Modeling

### XGBoost

In [111]:
XGB_model = xgb.XGBRegressor(n_estimators=300)

In [112]:
XGB_model.fit(X_train, y_train)

[22:20:18] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

### Predict and Save

In [113]:
y_pred = XGB_model.predict(X_test)

In [114]:
y_pred

array([ 3.5825462,  2.488554 , 15.929479 , ...,  3.2352824,  4.8687835,
       20.195084 ], dtype=float32)

In [115]:
y_pred_df = pd.DataFrame({'Price': y_pred})

In [116]:
y_pred_df.to_excel('Submissions/4_Predictions.xlsx', index=False)

# Experiment 5 - 0.9296

## Modeling

### XGBoost

In [132]:
# parameters = {
#     'n_estimators': [300, 500, 700],
#     'max_depth': range(3,10,2),
#     'min_child_weight': range(1,6,2),
#     'gamma': [i/10.0 for i in range(0,5)],
#     'subsample': [i/10.0 for i in range(6,10)],
#     'colsample_bytree': [i/10.0 for i in range(6,10)],
#     'learning_rate': [0.01, 0.1, 1],
#     'reg_alpha': [0, 0.001, 0.005, 0.01, 0.05, 0.1, 1]
# }

In [140]:
parameters = {
    'n_estimators': [350],
    'max_depth': [3,6,9],
    'min_child_weight': range(1,6,2),
    'gamma': [i/10.0 for i in range(0,3)],
    'subsample': [i/10.0 for i in range(6,8)],
    'colsample_bytree': [i/10.0 for i in range(6,8)],
    'learning_rate': [0.01, 0.1, 1],
    'reg_alpha': [0.01, 0.1, 1]
}

In [143]:
XGB_model_gird = GridSearchCV(estimator = xgb.XGBRegressor(n_seed=27), param_grid = parameters, n_jobs=4, cv=5, verbose=True)
XGB_model_gird.fit(X_train, y_train)

Fitting 5 folds for each of 972 candidates, totalling 4860 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   13.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  3.4min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  6.5min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:  9.9min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed: 14.2min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed: 19.6min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed: 25.9min
[Parallel(n_jobs=4)]: Done 4042 tasks      | elapsed: 33.3min
[Parallel(n_jobs=4)]: Done 4860 out of 4860 | elapsed: 40.3min finished


[09:52:29] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, n_seed=27,
                                    nthread=None, objective='reg:linear',
                                    rando...
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=4,
             param_grid={'colsample_bytree': [0.6, 0.7],
                         'gamma': [0.0, 0.1, 0.2],
                         'learning_rate'

In [144]:
XGB_model_gird.best_params_

{'colsample_bytree': 0.6,
 'gamma': 0.0,
 'learning_rate': 0.1,
 'max_depth': 9,
 'min_child_weight': 5,
 'n_estimators': 350,
 'reg_alpha': 0.1,
 'subsample': 0.6}

### Predict and Save

In [148]:
y_pred = XGB_model_gird.predict(X_test)

In [149]:
y_pred

array([ 3.5569546,  3.0236735, 16.704979 , ...,  3.3620975,  5.1493773,
       19.280022 ], dtype=float32)

In [150]:
y_pred_df = pd.DataFrame({'Price': y_pred})

In [151]:
y_pred_df.to_excel('Submissions/5_Predictions.xlsx', index=False)

- Increase number of estimators
- Fine tune the parameters

# Experiment 6 - 0.93

## Preprocessing

In [3]:
X_train = train_data.drop(columns='Price')
y_train = train_data['Price']

X_test = test_data.copy()

In [4]:
X_train['Engine'] = X_train['Engine'].str[:-3].replace(np.nan, -1).astype('int').replace(-1, np.nan)
X_test['Engine'] = X_test['Engine'].str[:-3].replace(np.nan, -1).astype('int').replace(-1, np.nan)

In [5]:
X_train['Power'] = X_train['Power'].str[:-4].replace(np.nan, -1).replace('null', -1).astype('float').replace(-1, np.nan)
X_test['Power'] = X_test['Power'].str[:-4].replace(np.nan, -1).replace('null', -1).astype('float').replace(-1, np.nan)

In [6]:
X_train['Mileage'] = X_train.Mileage.str.split(expand=True).drop(columns=1).rename(columns={0: 'Mileage'}).Mileage.astype('float')
X_test['Mileage'] = X_test.Mileage.str.split(expand=True).drop(columns=1).rename(columns={0: 'Mileage'}).Mileage.astype('float')

In [7]:
X_train.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.60,998.0,58.16,5.0,NaN
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,NaN
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.20,1199.0,88.70,5.0,8.61 Lakh
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,NaN
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,NaN


### Drop columns

In [8]:
X_train = X_train.drop(columns=['New_Price'])
X_test = X_test.drop(columns=['New_Price'])

In [9]:
X_train.isnull().sum()

Name                   0
Location               0
Year                   0
Kilometers_Driven      0
Fuel_Type              0
Transmission           0
Owner_Type             0
Mileage                2
Engine                36
Power                143
Seats                 42
dtype: int64

In [10]:
X_test.isnull().sum()

Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               0
Engine               10
Power                32
Seats                11
dtype: int64

### Impute missing values

In [11]:
X_train['Mileage'] = X_train.groupby(['Fuel_Type'])['Mileage'].transform(lambda x: x.fillna(x.mean()))

In [12]:
def fill_engine(x):
    return x.sum()/len(x)

X_train['Engine'] = X_train.groupby(['Name'])['Engine'].transform(lambda x: x.fillna(fill_engine(x)))

values = X_test[X_test['Engine'].isnull()]['Name'].values
X_test['Engine'].ix[X_test['Engine'].isnull()] = list(X_train.groupby('Name')['Engine'].mean()[values].values)

X_test['Engine'] = X_test['Engine'].fillna(X_train['Engine'].mean())

In [13]:
def fill_power(x):
    return x.sum()/len(x)

X_train['Power'] = X_train.groupby(['Name'])['Power'].transform(lambda x: x.fillna(fill_power(x)))

values = X_test[X_test['Power'].isnull()]['Name'].values
X_test['Power'].ix[X_test['Power'].isnull()] = list(X_train.groupby('Name')['Power'].mean()[values].values)

X_test['Power'] = X_test['Power'].fillna(X_train['Power'].mean())

In [14]:
def fill_seats(x, X_train):
    try:
        return x.mode()[0]
    except:
        return X_train['Seats'].mode()[0]
    
X_train['Seats'] = X_train.groupby(['Name'])['Seats'].transform(lambda x: x.fillna(fill_seats(x, X_train)))
X_test['Seats'] = X_test['Seats'].fillna(X_train['Seats'].mode()[0])

### Label Encoding

In [15]:
object_columns = ['Name', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type']

for col in object_columns:
    label_encoder = le.fit(list(X_train[col]) + list(X_test[col]))
    X_train[col] = label_encoder.transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])

## Modeling

### XGBoost

In [17]:
XGB_model = xgb.XGBRegressor(n_estimators=600, colsample_bytree=0.6, gamma=0.0, learning_rate=0.1, 
                             max_depth=9, min_child_weight=5, reg_alpha=0.1, subsample=0.6)

In [18]:
XGB_model.fit(X_train, y_train)

[22:48:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0.0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=9, min_child_weight=5, missing=None, n_estimators=600,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.6, verbosity=1)

### Predict and Save

In [19]:
y_pred = XGB_model.predict(X_test)

In [20]:
y_pred

array([ 3.3438115,  3.0888045, 16.680012 , ...,  3.3884344,  5.5023565,
       19.798151 ], dtype=float32)

In [21]:
y_pred_df = pd.DataFrame({'Price': y_pred})

In [23]:
y_pred_df.to_excel('Submissions/6_Predictions.xlsx', index=False)

# Experiment 7 - 0.9326	

## Preprocessing

In [91]:
X_train = train_data.drop(columns='Price')
y_train = train_data['Price']

X_test = test_data.copy()

In [92]:
X_train['Engine'] = X_train['Engine'].str[:-3].replace(np.nan, -1).astype('int').replace(-1, np.nan)
X_test['Engine'] = X_test['Engine'].str[:-3].replace(np.nan, -1).astype('int').replace(-1, np.nan)

In [93]:
X_train['Power'] = X_train['Power'].str[:-4].replace(np.nan, -1).replace('null', -1).astype('float').replace(-1, np.nan)
X_test['Power'] = X_test['Power'].str[:-4].replace(np.nan, -1).replace('null', -1).astype('float').replace(-1, np.nan)

In [94]:
X_train['Mileage'] = X_train.Mileage.str.split(expand=True).drop(columns=1).rename(columns={0: 'Mileage'}).Mileage.astype('float')
X_test['Mileage'] = X_test.Mileage.str.split(expand=True).drop(columns=1).rename(columns={0: 'Mileage'}).Mileage.astype('float')

In [95]:
X_train.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.60,998.0,58.16,5.0,NaN
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,NaN
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.20,1199.0,88.70,5.0,8.61 Lakh
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,NaN
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,NaN


### Drop columns

In [96]:
X_train = X_train.drop(columns=['New_Price'])
X_test = X_test.drop(columns=['New_Price'])

In [97]:
X_train.isnull().sum()

Name                   0
Location               0
Year                   0
Kilometers_Driven      0
Fuel_Type              0
Transmission           0
Owner_Type             0
Mileage                2
Engine                36
Power                143
Seats                 42
dtype: int64

In [98]:
X_test.isnull().sum()

Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               0
Engine               10
Power                32
Seats                11
dtype: int64

### Impute missing values

In [99]:
X_train['Mileage'] = X_train.groupby(['Fuel_Type'])['Mileage'].transform(lambda x: x.fillna(x.mean()))

In [100]:
def fill_engine(x):
    return x.sum()/len(x)

X_train['Engine'] = X_train.groupby(['Name'])['Engine'].transform(lambda x: x.fillna(fill_engine(x)))

values = X_test[X_test['Engine'].isnull()]['Name'].values
X_test['Engine'].ix[X_test['Engine'].isnull()] = list(X_train.groupby('Name')['Engine'].mean()[values].values)

X_test['Engine'] = X_test['Engine'].fillna(X_train['Engine'].mean())

In [101]:
def fill_power(x):
    return x.sum()/len(x)

X_train['Power'] = X_train.groupby(['Name'])['Power'].transform(lambda x: x.fillna(fill_power(x)))

values = X_test[X_test['Power'].isnull()]['Name'].values
X_test['Power'].ix[X_test['Power'].isnull()] = list(X_train.groupby('Name')['Power'].mean()[values].values)

X_test['Power'] = X_test['Power'].fillna(X_train['Power'].mean())

In [102]:
def fill_seats(x, X_train):
    try:
        return x.mode()[0]
    except:
        return X_train['Seats'].mode()[0]
    
X_train['Seats'] = X_train.groupby(['Name'])['Seats'].transform(lambda x: x.fillna(fill_seats(x, X_train)))
X_test['Seats'] = X_test['Seats'].fillna(X_train['Seats'].mode()[0])

### Label Encoding

In [103]:
object_columns = ['Name', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type']

for col in object_columns:
    label_encoder = le.fit(list(X_train[col]) + list(X_test[col]))
    X_train[col] = label_encoder.transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])

### Create new features

In [104]:
X_train.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
0,1310,9,2010,72000,0,1,0,26.60,998.0,58.16,5.0
1,560,10,2015,41000,1,1,0,19.67,1582.0,126.20,5.0
2,529,2,2011,46000,4,1,0,18.20,1199.0,88.70,5.0
3,1160,2,2012,87000,1,1,0,20.77,1248.0,88.76,7.0
4,23,3,2013,40670,1,0,2,15.20,1968.0,140.80,5.0


In [105]:
X_train['number_of_years_old'] = 2019 - X_train['Year']

X_train['E/M'] = X_train['Engine']/X_train['Mileage']
X_train['E/P'] = X_train['Engine']/X_train['Power']
X_train['E/(P+M)'] = X_train['Engine']/(X_train['Power'] + X_train['Mileage'])

In [106]:
X_test['number_of_years_old'] = 2019 - X_test['Year']

X_test['E/M'] = X_test['Engine']/X_test['Mileage']
X_test['E/P'] = X_test['Engine']/X_test['Power']
X_test['E/(P+M)'] = X_test['Engine']/(X_test['Power'] + X_test['Mileage'])

In [107]:
X_train.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,number_of_years_old,E/M,E/P,E/(P+M)
0,1310,9,2010,72000,0,1,0,26.60,998.0,58.16,5.0,9,37.518797,17.159560,11.774422
1,560,10,2015,41000,1,1,0,19.67,1582.0,126.20,5.0,4,80.427046,12.535658,10.845273
2,529,2,2011,46000,4,1,0,18.20,1199.0,88.70,5.0,8,65.879121,13.517475,11.216090
3,1160,2,2012,87000,1,1,0,20.77,1248.0,88.76,7.0,7,60.086663,14.060388,11.394139
4,23,3,2013,40670,1,0,2,15.20,1968.0,140.80,5.0,6,129.473684,13.977273,12.615385


## Modeling

### XGBoost

In [82]:
XGB_model = xgb.XGBRegressor(n_estimators=600, colsample_bytree=0.6, gamma=0.0, learning_rate=0.1, 
                             max_depth=9, min_child_weight=5, reg_alpha=0.1, subsample=0.6)

In [83]:
XGB_model.fit(X_train, y_train)

[23:31:46] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0.0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=9, min_child_weight=5, missing=None, n_estimators=600,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.6, verbosity=1)

### Predict and Save

In [108]:
y_pred = XGB_model.predict(X_test)

In [109]:
y_pred

array([ 3.075576 ,  3.1948016, 17.861397 , ...,  3.0230684,  4.8441296,
       20.242058 ], dtype=float32)

In [110]:
y_pred_df = pd.DataFrame({'Price': y_pred})

In [111]:
y_pred_df.to_excel('Submissions/7_Predictions.xlsx', index=False)

In [112]:
XGB_model.feature_importances_

array([0.01329779, 0.00764607, 0.0606748 , 0.01248724, 0.04601088,
       0.22428013, 0.00320112, 0.01456376, 0.09741119, 0.30645588,
       0.03907339, 0.06817382, 0.01367203, 0.07635164, 0.01670022],
      dtype=float32)

In [113]:
X_train.columns

Index(['Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type',
       'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats',
       'number_of_years_old', 'E/M', 'E/P', 'E/(P+M)'],
      dtype='object')

# Experiment 8 - 0.9326 (51)

In [114]:
X_train['E*M'] = X_train['Engine'] * X_train['Mileage']
X_test['E*M'] = X_test['Engine'] * X_test['Mileage']

## Modeling

### XGBoost

In [115]:
XGB_model = xgb.XGBRegressor(n_estimators=600, colsample_bytree=0.6, gamma=0.0, learning_rate=0.1, 
                             max_depth=9, min_child_weight=5, reg_alpha=0.1, subsample=0.6)

In [116]:
XGB_model.fit(X_train, y_train)

[23:43:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0.0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=9, min_child_weight=5, missing=None, n_estimators=600,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.6, verbosity=1)

### Predict and Save

In [117]:
y_pred = XGB_model.predict(X_test)

In [118]:
y_pred

array([ 3.1533697,  3.1481633, 18.106327 , ...,  3.0207229,  4.90435  ,
       21.391985 ], dtype=float32)

In [119]:
y_pred_df = pd.DataFrame({'Price': y_pred})

In [120]:
y_pred_df.to_excel('Submissions/8_Predictions.xlsx', index=False)

In [121]:
XGB_model.feature_importances_

array([0.01245112, 0.00794959, 0.05386748, 0.01211138, 0.04351325,
       0.17967393, 0.00312111, 0.01360628, 0.12006935, 0.3240982 ,
       0.04174773, 0.06139607, 0.01287523, 0.07557096, 0.01673083,
       0.02121748], dtype=float32)

In [122]:
X_train.columns

Index(['Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type',
       'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats',
       'number_of_years_old', 'E/M', 'E/P', 'E/(P+M)', 'E*M'],
      dtype='object')